# Hessian-free optimization

In [1]:
import sys, os
import numpy as np

In [4]:
# file paths to call
IR2_file = np.loadtxt("./test_data/IR_2.txt")
aerinite_file = np.loadtxt("./test_data/aerinite_processed.txt", delimiter=",")

IR_x = IR2_file[:,0]
IR_y = IR2_file[:,1]

aerinite_x = aerinite_file[:,0]
aerinite_y = aerinite_file[:,1]

First define the Gaussians and its partial derivatives with respect to its parameters.

In [5]:
def gaussian(x, I, mu, sigma):
    diff = x - mu
    dist = diff**2
    exponential = np.exp(-dist/(2*sigma**2 + 1e-11))
    amplitude = np.abs(I)/(np.sqrt(2*np.pi) * sigma)
    return amplitude * exponential

In [7]:
def gaussian_partial_I(x, I, mu, sigma):
    diff = x - mu
    dist = diff**2
    exponential = np.exp(-dist/(2*sigma**2 + 1e-11))
    amplitude = np.sign(I)/(np.sqrt(2*np.pi) * sigma)
    return amplitude * exponential

In [8]:
def gaussian_partial_mu(x, I, mu, sigma):
    diff = x - mu
    dist = diff**2
    exponential = np.exp(-dist/(2*sigma**2 + 1e-11))
    amplitude = I*(x-mu)/(np.sqrt(2*np.pi) * sigma**3)
    return amplitude * exponential

In [9]:
def gaussian_partial_sigma(x, I, mu, sigma):
    diff = x - mu
    dist = diff**2
    exponential = np.exp(-dist/(2*sigma**2 + 1e-11))
    amplitude = I*((x-mu)**2/sigma**2 - 1)/(np.sqrt(2*np.pi) * sigma**2)
    return amplitude * exponential

## Helper functions for the conjugate gradient method

Essentially we'd like to update $f(\mathbf{x}+\Delta \mathbf{x})$ by $\mathbf{x}_i = \mathbf{x}_{i-1} - \alpha_{i-1}\nabla f(\mathbf{x}_{i-1})$ with
$$
    \alpha_{i} = - \frac{[{\mathbf{x}_{i}}^T H(\mathbf{x}_{i}) + \nabla f(\mathbf{x}_{i})]\mathbf{d}_{i}}{{\mathbf{d}_{i}}^T H(\mathbf{x}_{i})\mathbf{d}_{i}}
$$
using the following approximation
$$
    f(\mathbf{x}+\Delta \mathbf{x}) \approx f(\mathbf{x}) + \nabla f(\mathbf{x})^T \Delta \mathbf{x} + \frac{1}{2} \Delta \mathbf{x}^T H(f) \Delta \mathbf{x}.
$$
where each row the Hessian can be approximated as a directional derivative in direction 
$$
    \mathbf{d}_{i} = -\nabla f(x_i) + \beta_{i-1} \mathbf{d}_{i-1} = - \nabla f(\mathbf{x}_{i-1}) + \frac{\nabla f(\mathbf{x}_{i})^T H(\mathbf{x}_{i-1}) \mathbf{d}_{i-1}}{{\mathbf{d}_{i-1}}^T H(\mathbf{x}_{i-1})\mathbf{d}_{i-1}}\mathbf{d}_{i-1}.
$$

The Hessian is quite horrid to compute, but its product with an arbitrary vector can be approximated as:
$$
    H\mathbf{v} \approx \frac{\nabla f(\mathbf{x}+\epsilon \mathbf{v}) - \nabla f(\mathbf{x})}{\epsilon}
$$
for sufficiently small $\epsilon > 0$.